# ToolNode 를 사용한 도구 사용기능 만들기
- 계산기, 날씨, 환율 도구

# 사전 설치 필요

In [ ]:
# 설치 필요
# pip install geopy 

from geopy.geocoders import Nominatim

# Nominatim geocoding class from the geopy library. 
# 주소를 좌표(위도, 경도)  혹은 좌표를 주소로 변환해두는 geocoding 라이브러리
# This class allows developers to convert addresses into geographic coordinates (latitude and longitude) and vice versa. 

# import

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import httpx   # http 클라이언트 라이브러리 (비동기/동기 요청 모두 가능.)
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain.chat_models import init_chat_model
import math
from geopy.geocoders import Nominatim
from IPython.display import Image, display

# 도구들 Tools

## 계산기 도구

In [3]:
"10 + 20"

'10 + 20'

In [4]:
eval("10 + 20")

30

In [5]:
"sqrt(2)"

'sqrt(2)'

In [7]:
eval("math.sqrt(2)")

1.4142135623730951

In [8]:
def calculator(expression: str) -> str:
    """수학 계산을 수행합니다."""
    print(f"계산 요청: {expression}")

    try:
        # 간단한 치횐
        expression.replace("sqrt", "math.sqrt")
        expression.replace("sin", "math.sin")
        expression.replace("cos", "math.cos")
    
        # 계산 실행
        # ※ eval() 은 파이썬 코드를 모드 실행할수 있기에, 위험한 실행동작을 예방하기 위해
        #    math 패키지만 사용하도록 제한.
        result = eval(expression, {"__builtins__": {}, "math": math})
        return f"계산 결과: {result}"

    except Exception as e:
        return f"계산 오류: {str(e)}"
        

## 날씨 정보 도구

In [13]:
def get_coordinates(city_name: str) -> tuple[float, float]:
    """도시 이름을 받아 위도와 경도를 반환합니다"""
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"Could not find coordinate for {city_name}")

# 테스트
# get_coordinates("비키니시티")

In [14]:
def get_weather(city_name: str) -> dict:
    """도시 이름을 받아 해당 도시의 현재 날씨 정보를 반환합니다."""

    if city_name:
        latitude, longitude = get_coordinates(city_name)
    else:
        raise ValueError("City name must be provided to get weather information.")

    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true" 
    response = httpx.get(url)    
    response.raise_for_status()  # HTTP 에러시 예외 발생
    return response.json()

# 테스트
# get_weather("서울")

{'latitude': 37.55,
 'longitude': 127.0,
 'generationtime_ms': 0.091552734375,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 34.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2026-01-06T12:30',
  'interval': 900,
  'temperature': 0.1,
  'windspeed': 4.3,
  'winddirection': 175,
  'is_day': 0,
  'weathercode': 1}}

## 환율 계산 도구

In [15]:
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    "통화 간 환율을 계산합니다"
    print(f"{amount} {from_currency}를 {to_currency}로 변환합니다")
    rates = {("USD", "KRW"): 1450.50, ("KRW", "USD"): 0.00076}
    rate_key = (from_currency.upper(), to_currency.upper())

    if rate_key in rates:
        rate = rates[rate_key]
        converted = amount * rate
        return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

    return f"{amount} {from_currency} = {amount} {to_currency} (동일 통화)"
        
        
# 테스트
currency_converter(1000000, "krw", "usd")
    

1000000 krw를 usd로 변환합니다


'1000000 krw = 760.00 usd'